In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import wav_to_mel_spect
import os
import pandas as pd
from scipy.io import wavfile

In [2]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 10
SOUNDS =np.array(['yes','no','right','five','nine'])
#path_out = '/home/deiry/semilleroML/Voice_Recognition/data/mel-spectrograms/'
path_out = '/Users/deiry/ML/Voice_Recognition/data/mel-spectograms/'

In [3]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [4]:
def createDataset(path_word_speakerId):
    speakers =[]
    words = []
    spectrograms = []
    pos = 0

     #Encode etiqueta del path audio,speaker y la palabra
    leSpeaker = preprocessing.LabelEncoder()
    leWord =  preprocessing.LabelEncoder()

    #Encode
    leSpeaker.fit(path_word_speakerId[:,2])
    leWord.fit(path_word_speakerId[:,1])


    for p in path_word_speakerId:
        name = p[0].split('/')[-1].split('.')[0]
        word = p[1]
        speaker_id = p[-1]

        #Transformación del encoder
        labelSpeaker = leSpeaker.transform([speaker_id])
        labelWord= leWord.transform([word])     
        wav_path = '../'+p[0]
        spect = wavToMelSpect(wav_path, norm=True, pad=True) 
        spectrograms.append(spect)
        speakers.append(labelSpeaker[0])
        words.append(labelWord[0])
        displayMelSpectogram(spect, word, pos)
        pos+=1
        
    
    return spectrograms,speakers, words


def wavToMelSpect(wav_path,n_mels=120, norm=True, pad = True):
    name = wav_path.split('/')[-1].split('.')[0]
    y, sr = librosa.load(wav_path, duration=1.0)

    spect, _ = librosa.effects.trim(y)
    print("orignal ",spect.shape)
              
    spect = librosa.feature.melspectrogram(y=spect, sr=sr,n_mels=128,hop_length=512,n_fft=2048)
    print("mel ",spect.shape)
    if(pad):
        spect = wav_to_mel_spect.pad_audio(spect) 
        
    print("pad ", spect.shape)
    if(norm):      
        spect=sk.preprocessing.minmax_scale(spect, axis=0)      
  
    print("final", spect.shape)
    return spect

def displayMelSpectogram(spect, word , pos):
    plt.figure(figsize=(10, 4))
    S_dB = librosa.power_to_db(spect, ref=np.max)
    librosa.display.specshow(S_dB, x_axis='time',
                             y_axis='mel', sr=22050,
                            fmax=8000)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-frequency spectrogram word '+ str(word) + '  position'+str(pos))
    plt.tight_layout()
    plt.show()

In [5]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [ ]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ['data/yes/00f0204f_nohash_0.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_1.wav', 'yes', '00f0204f'],
       ['data/yes/00f0204f_nohash_2.wav', 'yes', '00f0204f'],
       ['data/yes/012c8314_nohash_0.wav', 'yes', '012c8314'],
       ['data/yes/0132a06d_nohash_0.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_1.wav', 'yes', '0132a06d'],
       ['data/yes/0132a06d_nohash_2.wav', 'yes', '0132a06d'],
       ['data/no/012c8314_nohash_0.wav', 'no', '012c8314'],
       ['data/no/0132a06d_nohash_0.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_1.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_2.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_3.wav', 'no', '0132a06d'],
       ['data/no/0132a06d_nohash_4.wav', 'no', '0132a06d'],
       ['data/no/013

In [ ]:
nine = path_word_speakerId[-10:-1]

In [ ]:
nine.shape

(9, 3)

In [ ]:
nine

array([['data/nine/00b01445_nohash_0.wav', 'nine', '00b01445'],
       ['data/nine/012c8314_nohash_0.wav', 'nine', '012c8314'],
       ['data/nine/012c8314_nohash_1.wav', 'nine', '012c8314'],
       ['data/nine/012c8314_nohash_2.wav', 'nine', '012c8314'],
       ['data/nine/0132a06d_nohash_0.wav', 'nine', '0132a06d'],
       ['data/nine/0132a06d_nohash_1.wav', 'nine', '0132a06d'],
       ['data/nine/0132a06d_nohash_2.wav', 'nine', '0132a06d'],
       ['data/nine/0137b3f4_nohash_0.wav', 'nine', '0137b3f4'],
       ['data/nine/0137b3f4_nohash_1.wav', 'nine', '0137b3f4']],
      dtype='<U32')

In [ ]:
spectrograms,speakers, words = createDataset(nine)

orignal  (22050,)
mel  (128, 44)
pad  (22050, 21966)
final (22050, 21966)


In [ ]:
df = pd.DataFrame([spectrograms,speakers,words],index=['mel-spectrogram','speaker','word']).T
df.head()

In [ ]:
path_out = ''.join([path_out, 'mel-spectrograms_2500.hd5'])
path_out

In [ ]:
pwd
../data/mel-spectrograms_2500.hd5

In [ ]:
df.to_hdf('../data/mel-spectrograms_2500.hd5', key = 'description')

In [ ]:
#df.to_hdf('/Users/deiry/ML/Voice_Recognition/data/mel-spectograms/mel-spectrograms_7500.hd5', key = 'description')

In [ ]:
df_b = pd.read_hdf('../data/mel-spectrograms_2500.hd5')

In [ ]:
df_b.shape
df = df_b

In [ ]:
df_b.head()

In [ ]:
img = df['mel-spectrogram']
pos = 0
for i in range(0,len(img)):
    spect, word , pos = img[i], df['word'][i], i
    displayMelSpectogram(spect, word , pos)